In [1]:
import datetime
import glob
import math
import os

from pathlib import Path
from importlib import reload

import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import scipy.stats as stats

from plotly.subplots import make_subplots

from perf_model import PerfModel
from utils import *

# set the max columns to none
pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
results_dir = "../results"
plots_dir = "../plots"
perf_model_path = "../data/perf_model.csv"

In [5]:
#model_to_plot = "llama2-70b"
model_to_plot = "bloom-176b"
#seed = 9
seed = 0

In [6]:
perf_model = PerfModel(perf_model_path, init=True)
normalize_model = model_to_plot
normalize_hardware = "a100-80gb"
normalize_tp = 8

In [ ]:
# isopower
print("Num servers in cluster")

print("isopower")
max_h100 = 40
for num_h100 in range(0, max_h100+1, 5):
    num_a100 = int(44 * (max_h100 - num_h100) // 24.8)
    print(num_a100, num_h100, num_a100+num_h100)

# isocost
print("isocost")
max_h100 = 40
for num_h100 in range(0, max_h100+1, 5):
    num_a100 = int(4.76 * (max_h100 - num_h100) // 2.21)
    print(num_a100, num_h100, num_a100+num_h100)

print("isopower hhcap")
max_h100 = 40
for prompt_h100 in range(0, max_h100+1, 5):
    token_h100 = int(44 * (max_h100 - prompt_h100) // 30.8)
    print(prompt_h100, token_h100, prompt_h100+token_h100)


In [7]:
def get_data(configs, traces, seed, quantiles=[0.5, 0.9, 0.99], model=""):
    results = []
    request_dfs = {}
    for trace in traces:
        for config in configs:
            name = config["name"]
            scheduler = config["scheduler"]
            start_state = config["start_state"]
            cluster = config["cluster"]

            summary_df = get_summary_data(results_dir, scheduler, start_state, cluster, trace, seed, model=model)
            request_df = get_request_data(results_dir, scheduler, start_state, cluster, trace, seed, model=model)
            if summary_df is None or request_df is None:
                continue

            perf_model.add_baseline_perf(request_df, normalize_model, normalize_hardware, normalize_tp)
            request_df["baseline_e2e"] = request_df["baseline_ttft"] + request_df["baseline_tbt"] * (request_df["token_sizes"] - 1)
            request_df["ttft_slowdown"] = request_df["ttft_times"] / request_df["baseline_ttft"]
            request_df["tbt_slowdown"] = request_df["tbt_times"] / request_df["baseline_tbt"]
            request_df["e2e_slowdown"] = request_df["response_times"] / request_df["baseline_e2e"]

            # check if OOM by existence of oom.csv
            oom = False
            if os.path.exists(f"{results_dir}/{seed}/{start_state}/{trace}/{cluster}/{model}/{scheduler}/oom.csv"):
                oom = True

            result = {}
            for key, value in config.items():
                result[key] = value
            result["trace"] = trace
            result["seed"] = seed
            for quantile in quantiles:
                result[f"ttft_slowdown_p{int(quantile * 100)}"] = request_df["ttft_slowdown"].quantile(quantile)
                result[f"tbt_slowdown_p{int(quantile * 100)}"] = request_df["tbt_slowdown"].quantile(quantile)
                result[f"e2e_slowdown_p{int(quantile * 100)}"] = request_df["e2e_slowdown"].quantile(quantile)
            for quantile in quantiles:
                result[f"ttft_times_p{int(quantile * 100)}"] = summary_df[f"ttft_times_p{int(quantile * 100)}"][0]
                result[f"tbt_times_p{int(quantile * 100)}"] = summary_df[f"tbt_times_p{int(quantile * 100)}"][0]
                result[f"e2e_times_p{int(quantile * 100)}"] = summary_df[f"response_times_p{int(quantile * 100)}"][0]
            result["oom"] = oom

            # save results to later create a dataframe
            results.append(result)
            request_dfs[f"{name}_{trace}"] = request_df

    results_df = pd.DataFrame(results)
    return results_df, request_dfs

def get_slo(y_var, quantile):
    if y_var == "tbt_slowdown" or y_var == "e2e_slowdown":
        if quantile == 0.5:
            return 1.25
        if quantile == 0.9:
            return 1.5
        if quantile == 0.99:
            return 5
    elif y_var == "ttft_slowdown":
        if quantile == 0.5:
            return 2
        if quantile == 0.9:
            return 3
        if quantile == 0.99:
            return 6
    else:
        raise Exception(f"Invalid y_var:quantile {y_var}:{quantile}")

def get_y_limits(y_var, quantile):
    if quantile == 0.5 or quantile == 0.9:
        return {
            'bottom': 0,
            'top': 4
        }
    elif quantile == 0.99:
        return {
            'bottom': 0,
            'top': 8
        }
    raise Exception(f"Invalid y_var:quantile {y_var}:{quantile}")   

def plot_y_vs_trace_new(results_df,
                        traces,
                        y_vars=["ttft_times", "tbt_times", "e2e_times"],
                        y_vars_labels=["TTFT", "TBT", "E2E"],
                        quantiles=[0.5, 0.9, 0.99],
                        title=None):
    fig, axs = plt.subplots(nrows=len(y_vars),
                            ncols=len(quantiles),
                            figsize=(len(quantiles) * 2.5, len(y_vars) * 1.5),
                            sharex=True,
                            constrained_layout=True)

    # plot
    for y_var in y_vars:
        for quantile in quantiles:
            sns.lineplot(data=results_df,
                         x="trace",
                         y=f"{y_var}_p{int(quantile * 100)}",
                         hue="name",
                         style="name",
                         markers=True,
                         markersize=7,
                         ax=axs[y_vars.index(y_var)][quantiles.index(quantile)])

    for ax in axs.flatten():
        ax.grid()
        ax.get_legend().set_visible(False)
        ax.set_xlabel("Request Rate (req/s)")
        xlabels = [trace.split("_")[2] for trace in traces]
        ax.set_xticks(ticks=range(0, len(traces)), labels=xlabels)

    # create a single legend in center of figure
    handles, labels = axs[0][0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=3, title="", bbox_to_anchor=(0.5, 1.01))

    #axs[0][0].set_yscale("log")
    #axs[0][1].set_yscale("log")
    #axs[2][0].set_yscale("log")
    #axs[2][1].set_yscale("log")

    for y_var in y_vars:
        for quantile in quantiles:
            axs[y_vars.index(y_var)][quantiles.index(quantile)].set_ylabel(
                f"Normalized\np{int(quantile*100)} {y_vars_labels[y_vars.index(y_var)]}")
            #axs[y_vars.index(y_var)][quantiles.index(quantile)].set_ylabel(f"p{int(quantile*100)} {y_var} (s)")
            y_limits = get_y_limits(y_var, quantile)
            axs[y_vars.index(y_var)][quantiles.index(quantile)].set_ylim(**y_limits)
            slo = get_slo(y_var, quantile)
            # add SLO lines
            axs[y_vars.index(y_var)][quantiles.index(quantile)].axhline(y=slo, color="red", linestyle="--")

    if title:
        fig.suptitle(title)

    plt.margins(x=0)
    # set 300dpi
    plt.gcf().set_dpi(300)


def plot_y_vs_trace_new_swapped(results_df,
                                traces,
                                y_vars=["ttft_times", "tbt_times", "e2e_times"],
                                y_vars_labels=["TTFT", "TBT", "E2E"],
                                quantiles=[0.5, 0.9, 0.99],
                                title=None):
    fig, axs = plt.subplots(nrows=len(quantiles),
                            ncols=len(y_vars),
                            figsize=(len(y_vars) * 2.5, len(quantiles) * 1.5),
                            sharex=True,
                            constrained_layout=True)

    axs = np.array(axs).reshape(len(quantiles), len(y_vars))

    # plot
    for y_var in y_vars:
        for quantile in quantiles:
            sns.lineplot(data=results_df,
                         x="trace",
                         y=f"{y_var}_p{int(quantile * 100)}",
                         hue="name",
                         style="name",
                         markers=True,
                         markersize=7,
                         ax=axs[quantiles.index(quantile)][y_vars.index(y_var)])

    for ax in axs.flatten():
        ax.grid()
        ax.get_legend().set_visible(False)
        ax.set_xlabel("Request Rate (req/s)")
        xlabels = [trace.split("_")[2] for trace in traces]
        ax.set_xticks(ticks=range(0, len(traces)), labels=xlabels)

    # create a single legend in center of figure
    handles, labels = axs[0][0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=3, title="", bbox_to_anchor=(0.5, 1.01))

    #axs[0][0].set_yscale("log")
    #axs[0][1].set_yscale("log")
    #axs[2][0].set_yscale("log")
    #axs[2][1].set_yscale("log")

    for y_var in y_vars:
        for quantile in quantiles:
            axs[quantiles.index(quantile)][y_vars.index(y_var)].set_ylabel(
                f"Normalized\np{int(quantile*100)} {y_vars_labels[y_vars.index(y_var)]}")
            #axs[y_vars.index(y_var)][quantiles.index(quantile)].set_ylabel(f"p{int(quantile*100)} {y_var} (s)")
            y_limits = get_y_limits(y_var, quantile)
            axs[quantiles.index(quantile)][y_vars.index(y_var)].set_ylim(**y_limits)
            slo = get_slo(y_var, quantile)
            # add SLO lines
            axs[quantiles.index(quantile)][y_vars.index(y_var)].axhline(y=slo, color="red", linestyle="--")

    fig.subplots_adjust(left=0)
    if title:
        fig.suptitle(title)

    # set 300dpi
    plt.gcf().set_dpi(300)


In [8]:
slos = {
    "ttft_slowdown_p99": get_slo("ttft_slowdown", 0.99),
    "tbt_slowdown_p99": get_slo("tbt_slowdown", 0.99),
    "e2e_slowdown_p99": get_slo("e2e_slowdown", 0.99),
    "ttft_slowdown_p90": get_slo("ttft_slowdown", 0.9),
    "tbt_slowdown_p90": get_slo("tbt_slowdown", 0.9),
    "e2e_slowdown_p90": get_slo("e2e_slowdown", 0.9),
    "ttft_slowdown_p50": get_slo("ttft_slowdown", 0.5),
    "tbt_slowdown_p50": get_slo("tbt_slowdown", 0.5),
    "e2e_slowdown_p50": get_slo("e2e_slowdown", 0.5),
}

In [ ]:
def plot_cost_power_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of cost and power for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find cheapest configs
    cheapest_configs = find_cheapest(configs_within_slo)

    # find least power configs
    least_power_configs = find_least_power(configs_within_slo)

    # normalize cost and power to baseline_system
    baseline_cost = cheapest_configs[cheapest_configs["system"] == baseline_system]["cost"].values[0]
    baseline_power = least_power_configs[least_power_configs["system"] == baseline_system]["power"].values[0]
    cheapest_configs["cost"] /= baseline_cost
    least_power_configs["power"] /= baseline_power

    # reshape the dataframes for seaborn
    cheapest_configs_melted = cheapest_configs.melt(id_vars='system', value_vars='cost', 
                            var_name='variable', 
                            value_name='value')
    least_power_configs_melted = least_power_configs.melt(id_vars='system', value_vars='power', 
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([cheapest_configs_melted, least_power_configs_melted])

    # rename cost and power to cheapest and least power
    combined_df["variable"] = combined_df["variable"].replace("cost", "Cost")
    combined_df["variable"] = combined_df["variable"].replace("power", "Power")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = results_df["system"].unique()
    # add a \n between the name and the number of servers
    #xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    # set minor ticks on y-axis
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')
    return combined_df



In [ ]:
def plot_isothroughput_cost_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of cost and power for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find cheapest configs
    cheapest_configs = find_cheapest(configs_within_slo)
    
    # pick unique config for each system
    cheapest_configs = cheapest_configs.drop_duplicates(subset=["system"])

    # normalize cost and power to baseline_system
    baseline_servers = cheapest_configs[cheapest_configs["system"] == baseline_system]["num_servers"].values[0]
    baseline_cost = cheapest_configs[cheapest_configs["system"] == baseline_system]["cost"].values[0]
    baseline_power = cheapest_configs[cheapest_configs["system"] == baseline_system]["power"].values[0]
    cheapest_configs["num_servers"] /= baseline_servers
    cheapest_configs["cost"] /= baseline_cost
    cheapest_configs["power"] /= baseline_power

    # reshape the dataframes for seaborn
    num_servers_melted = cheapest_configs.melt(id_vars='system', value_vars='num_servers', 
                            var_name='variable', 
                            value_name='value')
    cheapest_melted = cheapest_configs.melt(id_vars='system', value_vars='cost', 
                            var_name='variable', 
                            value_name='value')
    power_melted = cheapest_configs.melt(id_vars='system', value_vars='power', 
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([num_servers_melted, cheapest_melted, power_melted])

    # rename cost and power to cheapest and least power
    combined_df["variable"] = combined_df["variable"].replace("num_servers", "#Servers")
    combined_df["variable"] = combined_df["variable"].replace("cost", "Cost")
    combined_df["variable"] = combined_df["variable"].replace("power", "Power")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = cheapest_configs["name"].unique()
    # add a \n between the name and the number of servers
    xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # add a \n between the name and the number of servers
    #xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    # set minor ticks on y-axis
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')
    return combined_df




In [ ]:
def plot_isothroughput_power_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of cost and power for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find cheapest configs
    cheapest_configs = find_least_power(configs_within_slo)
    
    # pick unique config for each system
    cheapest_configs = cheapest_configs.drop_duplicates(subset=["system"])

    # normalize cost and power to baseline_system
    baseline_servers = cheapest_configs[cheapest_configs["system"] == baseline_system]["num_servers"].values[0]
    baseline_cost = cheapest_configs[cheapest_configs["system"] == baseline_system]["cost"].values[0]
    baseline_power = cheapest_configs[cheapest_configs["system"] == baseline_system]["power"].values[0]
    cheapest_configs["num_servers"] /= baseline_servers
    cheapest_configs["cost"] /= baseline_cost
    cheapest_configs["power"] /= baseline_power

    # reshape the dataframes for seaborn
    num_servers_melted = cheapest_configs.melt(id_vars='system', value_vars='num_servers', 
                            var_name='variable', 
                            value_name='value')
    cheapest_melted = cheapest_configs.melt(id_vars='system', value_vars='cost', 
                            var_name='variable', 
                            value_name='value')
    power_melted = cheapest_configs.melt(id_vars='system', value_vars='power', 
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([num_servers_melted, cheapest_melted, power_melted])

    # rename cost and power to cheapest and least power
    combined_df["variable"] = combined_df["variable"].replace("num_servers", "#Servers")
    combined_df["variable"] = combined_df["variable"].replace("cost", "Cost")
    combined_df["variable"] = combined_df["variable"].replace("power", "Power")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = cheapest_configs["name"].unique()
    # add a \n between the name and the number of servers
    xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # add a \n between the name and the number of servers
    #xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    # set minor ticks on y-axis
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')
    return combined_df


In [ ]:
def plot_isothroughput_count_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of cost and power for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find cheapest configs
    cheapest_configs = find_least_count(configs_within_slo)

    # pick unique config for each system
    cheapest_configs = cheapest_configs.drop_duplicates(subset=["system"])

    # normalize cost and power to baseline_system
    baseline_servers = cheapest_configs[cheapest_configs["system"] == baseline_system]["num_servers"].values[0]
    baseline_cost = cheapest_configs[cheapest_configs["system"] == baseline_system]["cost"].values[0]
    baseline_power = cheapest_configs[cheapest_configs["system"] == baseline_system]["power"].values[0]
    cheapest_configs["num_servers"] /= baseline_servers
    cheapest_configs["cost"] /= baseline_cost
    cheapest_configs["power"] /= baseline_power

    # reshape the dataframes for seaborn
    num_servers_melted = cheapest_configs.melt(id_vars='system', value_vars='num_servers', 
                            var_name='variable', 
                            value_name='value')
    cheapest_melted = cheapest_configs.melt(id_vars='system', value_vars='cost', 
                            var_name='variable', 
                            value_name='value')
    power_melted = cheapest_configs.melt(id_vars='system', value_vars='power', 
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([num_servers_melted, cheapest_melted, power_melted])

    # rename cost and power to cheapest and least power
    combined_df["variable"] = combined_df["variable"].replace("num_servers", "#Servers")
    combined_df["variable"] = combined_df["variable"].replace("cost", "Cost")
    combined_df["variable"] = combined_df["variable"].replace("power", "Power")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = cheapest_configs["name"].unique()
    # add a \n between the name and the number of servers
    xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # add a \n between the name and the number of servers
    #xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    # set minor ticks on y-axis
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')
    return combined_df


In [ ]:
def plot_cost_throughput_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of cost and throughput for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find least power configs
    max_throughput_configs = find_max_throughput(configs_within_slo)
    #print(max_throughput_configs)

    # normalize cost and throughput to baseline_system
    baseline_servers = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["num_servers"].values[0]
    baseline_cost = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["cost"].values[0]
    baseline_throughput = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["throughput"].values[0]
    max_throughput_configs["num_servers"] /= baseline_servers
    max_throughput_configs["cost"] /= baseline_cost
    max_throughput_configs["throughput"] /= baseline_throughput

    # reshape the dataframes for seaborn
    max_throughput_melted = max_throughput_configs.melt(id_vars='system', value_vars='throughput', 
                            var_name='variable', 
                            value_name='value')
    cost_melted = max_throughput_configs.melt(id_vars='system', value_vars='cost', 
                            var_name='variable', 
                            value_name='value')
    num_servers_melted = max_throughput_configs.melt(id_vars='system', value_vars='num_servers',
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([num_servers_melted, max_throughput_melted, cost_melted])

    # rename cost and throughput to cheapest and max throughput
    combined_df["variable"] = combined_df["variable"].replace("cost", "Cost")
    combined_df["variable"] = combined_df["variable"].replace("throughput", "Throughput")
    combined_df["variable"] = combined_df["variable"].replace("num_servers", "#Servers")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = combined_df["name"].unique()
    # add a \n between the name and the number of servers
    xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    # set minor ticks on y-axis
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')
    return combined_df



In [ ]:
def plot_power_throughput_optimal_within_slo(results_df, baseline_system, slos):
    """
    plot grouped bar chart of power and throughput for each system,
    normalized to baseline_system
    """
    # find configs within SLO
    configs_within_slo = find_within_slo(results_df, slos)

    # find least power configs
    max_throughput_configs = find_max_throughput(configs_within_slo)
    #print(max_throughput_configs)

    # normalize power and throughput to baseline_system
    baseline_servers = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["num_servers"].values[0]
    baseline_power = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["power"].values[0]
    baseline_throughput = max_throughput_configs[max_throughput_configs["system"] == baseline_system]["throughput"].values[0]
    max_throughput_configs["num_servers"] /= baseline_servers
    max_throughput_configs["power"] /= baseline_power
    max_throughput_configs["throughput"] /= baseline_throughput

    # reshape the dataframes for seaborn
    max_throughput_melted = max_throughput_configs.melt(id_vars='system', value_vars='throughput', 
                            var_name='variable', 
                            value_name='value')
    power_melted = max_throughput_configs.melt(id_vars='system', value_vars='power', 
                            var_name='variable', 
                            value_name='value')
    num_servers_melted = max_throughput_configs.melt(id_vars='system', value_vars='num_servers',
                            var_name='variable', 
                            value_name='value')
    combined_df = pd.concat([num_servers_melted, max_throughput_melted, power_melted])

    # rename power and throughput to cheapest and max throughput
    combined_df["variable"] = combined_df["variable"].replace("power", "Power")
    combined_df["variable"] = combined_df["variable"].replace("throughput", "Throughput")
    combined_df["variable"] = combined_df["variable"].replace("num_servers", "#Servers")

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)
    # order baseline_system first
    sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark', ax=ax)
                #order=[baseline_system, "Baseline-A100", "Splitwise-AA", "Splitwise-HA", "Splitwise-HH", "Splitwise-HHcap"])
    #sns.barplot(data=combined_df, x='system', y='value', hue='variable', palette='dark')
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=3, title="")
    xticklabels = results_df["name"].unique()
    # add a \n between the name and the number of servers
    xticklabels = [f"{xticklabel.split('(')[0]}\n({xticklabel.split('(')[1]}" for xticklabel in xticklabels]
    # rotate and set xticks
    ax.set_xticks(ticks=range(0, len(results_df["system"].unique())), labels=xticklabels, rotation=90)
    ax.set_ylabel("Normalized Value")
    ax.set_xlabel("")
    ax.grid(axis='y')
    ax.minorticks_on()
    ax.set_axisbelow(True)
    # make minor ticks increment every 0.25
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black', axis='y')


In [ ]:
# isopower clusters, conv trace

isopower_conv_configs = [
    baseline_a100_config(70),
    baseline_h100_config(40),
    splitwise_aa_config(45, 25),
    splitwise_hh_config(25, 15),
    splitwise_ha_config(25, 26),
    splitwise_hhcap_config(25, 21),
]

min_x = 50
max_x = 181
if model_to_plot == "bloom-176b":
    min_x = 30
    max_x = 161
isopower_conv_traces = [f"rr_conv_{i}_1min" for i in range(80, 151, 10)]
isopower_conv_traces = [f"rr_conv_{i}" for i in range(min_x, max_x, 10)]

isopower_conv_results_df, isopower_conv_request_dfs = get_data(
    isopower_conv_configs, isopower_conv_traces, seed=seed, model=model_to_plot)
plot_y_vs_trace_new(isopower_conv_results_df,
                    isopower_conv_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    #y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="")
                    #title="Isopower Clusters, Conversation Trace")
axs = plt.gca()
axs.set_xticks(ticks=range(0, len(isopower_conv_traces), 2), labels=[f"{i}" for i in range(min_x, max_x, 20)])
#plt.subplots_adjust(left=0.)
isopower_conv_results_df.to_csv(f"{plotsdata_dir}/isopower_conv_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_conv_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# isopower clusters, code trace

isopower_code_configs = [
    baseline_a100_config(70),
    baseline_h100_config(40),
    splitwise_aa_config(55, 15),
    splitwise_hh_config(35, 5),
    splitwise_ha_config(35, 8),
    splitwise_hhcap_config(35, 7),
]

min_x = 50
max_x = 181
if model_to_plot == "bloom-176b":
    min_x = 30
    max_x = 101
isopower_code_traces = [f"rr_code_{i}_1min" for i in range(50, 121, 10)]
isopower_code_traces = [f"rr_code_{i}" for i in range(min_x, max_x, 10)]

isopower_code_results_df, isopower_code_request_dfs = get_data(
    isopower_code_configs, isopower_code_traces, seed=seed, model=model_to_plot)
plot_y_vs_trace_new(isopower_code_results_df,
                    isopower_code_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    #y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="")
                    #title="Isopower Clusters, Code Trace")
isopower_code_results_df.to_csv(f"{plotsdata_dir}/isopower_code_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_code_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# code cluster for conv trace

isopower_code_for_conv_results_df, isopower_code_for_conv_request_dfs = \
    get_data(isopower_code_configs, isopower_conv_traces, seed=seed, model=model_to_plot)
plot_y_vs_trace_new_swapped(isopower_code_for_conv_results_df,
                    isopower_conv_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    quantiles=[0.9],
                    #y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="")
axs = plt.gca()
min_x = 50
max_x = 181
if model_to_plot == "bloom-176b":
    min_x = 30 
    max_x = 161
axs.set_xticks(ticks=range(0, len(isopower_conv_traces), 2), labels=[f"{i}" for i in range(30, max_x, 20)])
#plt.savefig(f"{plots_dir}/isopower_code_for_conv_{model_to_plot}.pdf", bbox_inches='tight')
isopower_code_for_conv_results_df.to_csv(f"{plotsdata_dir}/isopower_code_for_conv_{model_to_plot}_p90.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_code_for_conv_{model_to_plot}_p90.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
find_max_throughput(find_within_slo(isopower_code_for_conv_results_df, slos))

In [ ]:
find_max_throughput(find_within_slo(isopower_conv_results_df, slos))

In [ ]:
# conv cluster for code trace

isopower_conv_for_code_results_df, isopower_conv_for_code_request_dfs = \
    get_data(isopower_conv_configs, isopower_code_traces, seed=seed, model=model_to_plot)
plot_y_vs_trace_new(isopower_conv_for_code_results_df,
                    isopower_code_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="Isopower Conversation Cluster, Code Trace")
max_x = 181
if model_to_plot == "bloom-176b":
    max_x = 141
isopower_conv_for_code_results_df.to_csv(f"{plotsdata_dir}/isopower_conv_for_code_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_conv_for_code_{model_to_plot}.pdf", bbox_inches='tight')
#axs.set_xticks(ticks=range(0, len(isopower_code_traces), 2), labels=[f"{i}" for i in range(50, max_x, 20)])
plt.show()
plt.close()

In [ ]:
find_max_throughput(find_within_slo(isopower_code_results_df, slos))[["name", "trace"]]

In [ ]:
find_max_throughput(find_within_slo(isopower_conv_for_code_results_df, slos))[["name", "trace"]]

In [ ]:
# isocost clusters, conv trace

isocost_conv_configs = [
    baseline_a100_config(86),
    baseline_h100_config(40),
    splitwise_aa_config(51, 35),
    splitwise_hh_config(25, 15),
    splitwise_ha_config(30, 21),
    splitwise_hhcap_config(30, 10),
]

max_x = 181
if model_to_plot == "bloom-176b":
    max_x = 171
isocost_conv_traces = [f"rr_conv_{i}_1min" for i in range(80, 151, 10)]
isocost_conv_traces = [f"rr_conv_{i}" for i in range(50, max_x, 10)]

isocost_conv_results_df, isocost_conv_request_dfs = get_data(
    isocost_conv_configs, isocost_conv_traces, seed=seed, model=model_to_plot)
plt.close()
plot_y_vs_trace_new(isocost_conv_results_df,
                    isocost_conv_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="Isocost Clusters, Conversation Trace")
axs = plt.gca()
axs.set_xticks(ticks=range(0, len(isocost_conv_traces), 2), labels=[f"{i}" for i in range(50, max_x, 20)])
isocost_conv_results_df.to_csv(f"{plotsdata_dir}/isocost_conv_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isocost_conv_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# isocost clusters, code trace

isocost_code_configs = [
    baseline_a100_config(86),
    baseline_h100_config(40),
    splitwise_aa_config(66, 20),
    splitwise_hh_config(35, 5),
    splitwise_ha_config(35, 10),
    splitwise_hhcap_config(35, 7),
]

max_x = 181
if model_to_plot == "bloom-176b":
    max_x = 121
isocost_code_traces = [f"rr_code_{i}_1min" for i in range(50, 121, 10)]
isocost_code_traces = [f"rr_code_{i}" for i in range(50, max_x, 10)]

isocost_code_results_df, isocost_code_request_dfs = get_data(
    isocost_code_configs, isocost_code_traces, seed=seed, model=model_to_plot)
plot_y_vs_trace_new(isocost_code_results_df,
                    isocost_code_traces,
                    y_vars=["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"],
                    y_vars_labels=["TTFT\nSlowdown", "TBT\nSlowdown", "E2E\nSlowdown"],
                    title="Isocost Clusters, Code Trace")
isocost_code_results_df.to_csv(f"{plotsdata_dir}/isocost_code_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isocost_code_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# cost and max.throughput for isopower configurations

plot_cost_throughput_optimal_within_slo(isopower_conv_results_df, "Baseline-A100", slos)
isopower_conv_results_df.to_csv(f"{plotsdata_dir}/isopower_conv_cost_throughput_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_conv_cost_throughput_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

plot_cost_throughput_optimal_within_slo(isopower_code_results_df, "Baseline-A100", slos)
isopower_code_results_df.to_csv(f"{plotsdata_dir}/isopower_code_cost_throughput_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isopower_code_cost_throughput_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# power and max. throughput for isopower configurations

plot_power_throughput_optimal_within_slo(isocost_conv_results_df, "Baseline-A100", slos)
isocost_conv_results_df.to_csv(f"{plotsdata_dir}/isocost_conv_power_throughput_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isocost_conv_power_throughput_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

plot_power_throughput_optimal_within_slo(isocost_code_results_df, "Baseline-A100", slos)
isocost_code_results_df.to_csv(f"{plotsdata_dir}/isocost_code_power_throughput_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isocost_code_power_throughput_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# provisioning sweep

configs = []

min_a100 = 1
max_a100 = 70
min_h100 = 1
max_h100 = 40

for num_a100 in range(min_a100, 2*max_a100, 1):
    configs.append(baseline_a100_config(num_a100))

for num_h100 in range(min_h100, 2*max_h100, 1):
    configs.append(baseline_h100_config(num_h100))

for num_prompts in range(min_a100, max_a100, 1):
    for num_tokens in range(min_a100, max_a100, 1):
        configs.append(splitwise_aa_config(num_prompts, num_tokens))

for num_prompts in range(min_h100, max_h100, 1):
    for num_tokens in range(min_h100, max_h100, 1):
        configs.append(splitwise_hh_config(num_prompts, num_tokens))

for num_prompts in range(min_h100, max_h100, 1):
    for num_tokens in range(min_a100, max_a100, 1):
        configs.append(splitwise_ha_config(num_prompts, num_tokens))

for num_prompts in range(min_h100, max_h100, 1):
    for num_tokens in range(min_h100, max_h100, 1):
        configs.append(splitwise_hhcap_config(num_prompts, num_tokens))

provision_code_trace = ["rr_code_70_2min"]
#provision_conv_trace = ["rr_conv_70_2min"]

provision_code_df, provision_code_request_dfs = get_data(configs, traces=provision_code_trace, seed=seed, model=model_to_plot)
#provision_conv_df, provision_conv_request_dfs = get_data(configs, traces=provision_conv_trace, seed=seed, model=model_to_plot)

In [ ]:
def plot_provision_sweep(result_df):
    subset_df = result_df

    # plot scatter of num_prompts vs num_tokens for the ttft times
    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(9, 7), constrained_layout=True)

    title_dict = {
        "ttft_slowdown": "TTFT Slowdown",
        "tbt_slowdown": "TBT Slowdown",
        "e2e_slowdown": "E2E Slowdown"
    }

    y_vars = ["ttft_slowdown", "tbt_slowdown", "e2e_slowdown"]
    quantiles = [0.5, 0.9, 0.99]

    cmap = plt.get_cmap('viridis_r')
    colors = cmap(np.arange(cmap.N))

    max_slowdown = max(subset_df["ttft_slowdown_p99"].max(), subset_df["tbt_slowdown_p99"].max(), subset_df["e2e_slowdown_p99"].max())
    max_slo = 6
    colors[-1] = [0, 0, 0, 1]  # RGBA for black
    new_cmap = mcolors.ListedColormap(colors)

    all_slo_df = subset_df
    for y_var in y_vars:
        for quantile in quantiles:
            all_slo_df = all_slo_df[all_slo_df[f"{y_var}_p{int(quantile * 100)}"] <= get_slo(y_var, quantile)]

    for y_var in y_vars:
        for quantile in quantiles:
            sc = axs[y_vars.index(y_var)][quantiles.index(quantile)].scatter(
                subset_df["num_prompts"],
                subset_df["num_tokens"],
                c=subset_df[f"{y_var}_p{int(quantile * 100)}"],
                #norm=mcolors.LogNorm(vmin=1, vmax=max_slo),
                norm=mcolors.Normalize(vmin=1, vmax=max_slo),
                cmap=new_cmap
                #cmap='viridis_r'
            )
            axs[y_vars.index(y_var)][quantiles.index(quantile)].set_xlabel("# Prompt Machines")
            axs[y_vars.index(y_var)][quantiles.index(quantile)].set_ylabel("# Token Machines")
            axs[y_vars.index(y_var)][quantiles.index(quantile)].set_title(f"p{int(quantile * 100)} {title_dict[y_var]}")
            #axs[y_vars.index(y_var)][quantiles.index(quantile)].legend(bbox_to_anchor=(0.5, 1.11), loc="lower center", ncol=3, fontsize=7)
            axs[y_vars.index(y_var)][quantiles.index(quantile)].grid()
            
            slo = get_slo(y_var, quantile)
            # Filter the DataFrame to only include rows that meet the SLO
            slo_df = subset_df[subset_df[f"{y_var}_p{int(quantile * 100)}"] <= slo]

            # Group by 'num_prompts' and find the minimum 'num_tokens' for each group
            frontier_df = slo_df.groupby('num_prompts')['num_tokens'].min().reset_index()

            # Plot the frontier points
            axs[y_vars.index(y_var)][quantiles.index(quantile)].scatter(
                frontier_df["num_prompts"],
                frontier_df["num_tokens"],
                c='red',  # Color of the frontier points
                label='SLO Frontier'
            )

            # Find the cost optimal and power optimal points that meet the SLO
            #cost_optimal_point = slo_df.loc[slo_df['cost'].idxmin()]
            #power_optimal_point = slo_df.loc[slo_df['power'].idxmin()]
            cost_optimal_point = all_slo_df.loc[all_slo_df['cost'].idxmin()]
            power_optimal_point = all_slo_df.loc[all_slo_df['power'].idxmin()]

            # Plot the cost optimal point
            axs[y_vars.index(y_var)][quantiles.index(quantile)].scatter(
                cost_optimal_point["num_prompts"],
                cost_optimal_point["num_tokens"],
                c='azure',  # Color of the cost optimal point
                s=120,
                label='Cost Optimal',
                marker='*',
                edgecolors='black',
            )

            # Plot the power optimal point
            #axs[y_vars.index(y_var)][quantiles.index(quantile)].scatter(
            #    power_optimal_point["num_prompts"],
            #    power_optimal_point["num_tokens"],
            #    c='green',  # Color of the power optimal point
            #    label='Power Optimal'
            #)

            #axs[y_vars.index(y_var)][quantiles.index(quantile)].legend()

            # add colorbar
            #plt.colorbar(sc, ax=axs[y_vars.index(y_var)][quantiles.index(quantile)])
            # add colorbar with custom tick labels
            cbar = plt.colorbar(sc, ax=axs[y_vars.index(y_var)][quantiles.index(quantile)])
            tick_locs = list(cbar.get_ticks())
            #tick_locs.append(slo)
            cbar.set_ticks(tick_locs)
            tick_labels = [str(int(tick)) if tick < max_slo else f'{max_slo}+' for tick in tick_locs]
            #tick_labels[-1] = 'SLO'
            cbar.set_ticklabels(tick_labels)
            cbar.ax.axhline(get_slo(y_var, quantile), color='red', linewidth=3)

            # Add SLO label to the left of the colorbar
            cbar.set_label('SLO', rotation=0, labelpad=0, color='red')
            slo_position = (slo - 1) / (max_slo - 1)
            cbar.ax.yaxis.set_label_coords(-1.7, slo_position + 0.05)
    print(cost_optimal_point["num_prompts"], cost_optimal_point["num_tokens"])


In [ ]:
plot_provision_sweep(provision_code_df[provision_code_df["system"] == "Splitwise-HH"])
plt.savefig(f"{plots_dir}/hh_provision_code_sweep_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
provision_code_df.to_csv(f"{plotsdata_dir}/provision_code_sweep_{model_to_plot}.csv", index=False)
provision_conv_df.to_csv(f"{plotsdata_dir}/provision_conv_sweep_{model_to_plot}.csv", index=False)

In [ ]:
isothroughput_code_cost_df = plot_isothroughput_cost_optimal_within_slo(provision_code_df, "Baseline-A100", slos)
isothroughput_code_cost_df.to_csv(f"{plotsdata_dir}/isothroughput_code_cost_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isothroughput_code_cost_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

isothroughput_code_power_df = plot_isothroughput_power_optimal_within_slo(provision_code_df, "Baseline-A100", slos)
isothroughput_code_power_df.to_csv(f"{plotsdata_dir}/isothroughput_code_power_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isothroughput_code_power_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

isothroughout_code_count_df = plot_isothroughput_count_optimal_within_slo(provision_code_df, "Baseline-A100", slos)
isothroughout_code_count_df.to_csv(f"{plotsdata_dir}/isothroughput_code_count_{model_to_plot}.csv", index=False)
plt.savefig(f"{plots_dir}/isothroughput_code_count_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
isothroughput_conv_cost_df = plot_isothroughput_cost_optimal_within_slo(provision_conv_df, "Baseline-A100", slos)
isothroughput_conv_cost_df.to_csv(f"{plotsdata_dir}/isothroughput_conv_cost_{model_to_plot}.csv")
plt.savefig(f"{plots_dir}/isothroughput_conv_cost_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

isothroughput_conv_power_df = plot_isothroughput_power_optimal_within_slo(provision_conv_df, "Baseline-A100", slos)
isothroughput_conv_power_df.to_csv(f"{plotsdata_dir}/isothroughput_conv_power_{model_to_plot}.csv")
plt.savefig(f"{plots_dir}/isothroughput_conv_power_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

isothroughput_conv_count_df = plot_isothroughput_count_optimal_within_slo(provision_conv_df, "Baseline-A100", slos)
isothroughput_conv_count_df.to_csv(f"{plotsdata_dir}/isothroughput_conv_count_{model_to_plot}.csv")
plt.savefig(f"{plots_dir}/isothroughput_conv_count_{model_to_plot}.pdf", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
find_least_count(find_within_slo(provision_conv_df, slos)).drop_duplicates(subset=["system"])

In [ ]:
find_least_power(find_within_slo(provision_code_df, slos))

In [ ]:
def find_throughput(request_nodes_df, t_start=60, t_end=540):
    """
    Finds the throughput of a request_df between t_start and t_end
    """
    return request_nodes_df[
            (request_nodes_df["node_type"] == "TOKEN") & \
            (request_nodes_df["completion_timestamp"] >= t_start) & \
            (request_nodes_df["completion_timestamp"] < t_end)]["request_id"].nunique() / (t_end - t_start)

In [ ]:
for config in isocost_conv_configs:
    request_nodes_df = get_request_nodes_with_config(results_dir, config, "rr_conv_180", seed=0, model=model_to_plot)
    throughput = find_throughput(request_nodes_df)
    print(f"{config['name']}: {throughput} rps, ${round(config['cost'], 2)}, {round(throughput/config['cost'], 2)} rps/$")


In [ ]:
configs_within_slo = find_within_slo(provision_code_df, slos)
cheapest_configs = find_cheapest(configs_within_slo)
least_power_configs = find_least_power(configs_within_slo)

In [ ]:
def plot_num_batch_tokens(config, trace, seed, model=""):
    df = get_instances_data_with_config(results_dir, config, trace, seed, model)
    if "splitwise" in config.get("start_state", ""):
        num_prompt_batch_tokens, num_token_batch_tokens = get_num_batch_tokens_splitwise(df)
        sns.ecdfplot(num_prompt_batch_tokens, label="Prompt")
        sns.ecdfplot(num_token_batch_tokens, label="Token")
    else:
        num_batch_tokens = get_num_batch_tokens(df)
        sns.ecdfplot(num_batch_tokens, label="Batch")
    plt.xlabel("Number of Batch Tokens")
    plt.ylabel("CDF")
    plt.legend()
    plt.title(f'{config["name"]} {config["scheduler"]}, {trace}')
    plt.grid()
    return df

In [ ]:
def plot_duration_ccdf_batch_tokens(config, trace, seed, model=""):
    df = get_instances_data_with_config(results_dir, config, trace, seed, model)
    grouped_df = df[["batch_tokens", "duration"]].groupby("batch_tokens").sum()

    # find idle durations between iteration_end and iteration_start of next row in df for each instance identified by name
    total_idle_duration = 0
    for name in df["name"].unique():
        inst_df = df[df["name"] == name].sort_values("iteration_start")
        inst_df["idle_duration"] = inst_df["iteration_start"] - inst_df["iteration_end"].shift(1)
        inst_df["idle_duration"] = inst_df["idle_duration"].fillna(0)
        total_idle_duration += inst_df["idle_duration"].sum()

    # add entry to grouped_df at index 0
    grouped_df.loc[0] = [total_idle_duration]

    # sortby grouped_df.index
    grouped_df = grouped_df.sort_index()
    grouped_df["cum_duration"] = np.cumsum(grouped_df["duration"])
    grouped_df["cum_duration"] /= grouped_df["cum_duration"].max()
    # plot cumulative cdf of duration vs number of batch tokens
    #plt.plot(grouped_df.index, np.cumsum(grouped_df["duration"]))
    plt.plot(grouped_df.index, grouped_df["cum_duration"])
    plt.xlabel("Number of Batch Tokens")
    plt.ylabel("Cumulative Duration (s)")
    plt.grid()
    plt.title(f'{config["name"]} {config["scheduler"]}, {trace}')

    #if "splitwise" in config.get("start_state", ""):
    #    n_prompt = int(config["start_state"].split("_")[1])
    #    num_prompt_batch_tokens, num_token_batch_tokens = get_num_batch_tokens_splitwise(df, n_prompt)
    #    sns.ecdfplot(num_prompt_batch_tokens, label="Prompt")
    #    sns.ecdfplot(num_token_batch_tokens, label="Token")
    #else:
    #    num_batch_tokens = get_num_batch_tokens(df)
    #    sns.ecdfplot(num_batch_tokens, label="Batch")
    #plt.xlabel("Number of Batch Tokens")
    #plt.ylabel("CDF")
    #plt.legend()
    #plt.grid()
    #return df   

In [ ]:
def find_idle_duration(df):
    total_idle_duration = 0
    for name in df["name"].unique():
        inst_df = df[df["name"] == name].sort_values("iteration_start")
        inst_df["idle_duration"] = inst_df["iteration_start"] - inst_df["iteration_end"].shift(1)
        inst_df["idle_duration"] = inst_df["idle_duration"].fillna(0)
        total_idle_duration += inst_df["idle_duration"].sum()
    return total_idle_duration

def compare_duration_ccdf_batch_tokens(baseline_config, splitwise_config, trace, seed, model=""):
    baseline_df = get_instances_data_with_config(results_dir, baseline_config, trace, seed, model)
    splitwise_df = get_instances_data_with_config(results_dir, splitwise_config, trace, seed, model)

    # for baseline
    # find idle durations between iteration_end and iteration_start of next row in df for each instance identified by name
    baseline_idle_duration = find_idle_duration(baseline_df)
    # add entry to grouped df at index 0
    grouped_baseline_df = baseline_df[["batch_tokens", "duration"]].groupby("batch_tokens").sum()
    grouped_baseline_df.loc[0] = [baseline_idle_duration]

    # for splitwise, separate into prompt and token servers, and repeat the same process
    # find idle durations between iteration_end and iteration_start of next row in df for each instance identified by name
    splitwise_prompt_df = splitwise_df[splitwise_df["tag"] == "prompt"]
    splitwise_token_df = splitwise_df[splitwise_df["tag"] == "token"]
    splitwise_prompt_idle_duration = find_idle_duration(splitwise_prompt_df)
    splitwise_token_idle_duration = find_idle_duration(splitwise_token_df)
    # add entry to grouped df at index 0
    grouped_splitwise_prompt_df = splitwise_prompt_df[["batch_tokens", "duration"]].groupby("batch_tokens").sum()
    grouped_splitwise_prompt_df.loc[0] = [splitwise_prompt_idle_duration]
    grouped_splitwise_token_df = splitwise_token_df[["batch_tokens", "duration"]].groupby("batch_tokens").sum()
    grouped_splitwise_token_df.loc[0] = [splitwise_token_idle_duration]

    # sortby grouped_df.index
    grouped_baseline_df = grouped_baseline_df.sort_index()
    grouped_baseline_df["cum_duration"] = np.cumsum(grouped_baseline_df["duration"])
    grouped_baseline_df["cum_duration"] /= grouped_baseline_df["cum_duration"].max()

    grouped_splitwise_prompt_df = grouped_splitwise_prompt_df.sort_index()
    grouped_splitwise_prompt_df["cum_duration"] = np.cumsum(grouped_splitwise_prompt_df["duration"])
    grouped_splitwise_prompt_df["cum_duration"] /= grouped_splitwise_prompt_df["cum_duration"].max()
    
    grouped_splitwise_token_df = grouped_splitwise_token_df.sort_index()
    grouped_splitwise_token_df["cum_duration"] = np.cumsum(grouped_splitwise_token_df["duration"])
    grouped_splitwise_token_df["cum_duration"] /= grouped_splitwise_token_df["cum_duration"].max()

    fig, axs = plt.subplots(1, 1, figsize=(3, 3), constrained_layout=True)

    # plot cumulative cdf of duration vs number of batch tokens
    #plt.plot(grouped_df.index, np.cumsum(grouped_df["duration"]))
    axs.plot(grouped_baseline_df.index, grouped_baseline_df["cum_duration"], label="Baseline-H100")
    axs.plot(grouped_splitwise_prompt_df.index, grouped_splitwise_prompt_df["cum_duration"], label="Splitwise-HH (Prompt)")
    axs.plot(grouped_splitwise_token_df.index, grouped_splitwise_token_df["cum_duration"], label="Splitwise-HH (Token)")

    axs.set_xlabel("Number of Batch Tokens")
    axs.set_ylabel("Cumulative Duration")
    axs.grid()
    axs.set_xscale("log")
    axs.legend(loc="lower center", ncol=1, bbox_to_anchor=(0.5, 1.0))
    #ax.title(f'{config["name"]} {config["scheduler"]}, {trace}')
    
    # return dataframe
    return baseline_df, splitwise_df

    #if "splitwise" in config.get("start_state", ""):
    #    n_prompt = int(config["start_state"].split("_")[1])
    #    num_prompt_batch_tokens, num_token_batch_tokens = get_num_batch_tokens_splitwise(df, n_prompt)
    #    sns.ecdfplot(num_prompt_batch_tokens, label="Prompt")
    #    sns.ecdfplot(num_token_batch_tokens, label="Token")
    #else:
    #    num_batch_tokens = get_num_batch_tokens(df)
    #    sns.ecdfplot(num_batch_tokens, label="Batch")
    #plt.xlabel("Number of Batch Tokens")
    #plt.ylabel("CDF")
    #plt.legend()
    #plt.grid()
    #return df   

In [ ]:
baseline_df, splitwise_df = compare_duration_ccdf_batch_tokens(
    baseline_h100_config(40),
    splitwise_hh_config(25, 15),
    #"rr_conv_70",
    "rr_conv_130",
    seed=0,
    model=model_to_plot)
baseline_df.to_csv(os.path.join(plotsdata_dir, f"duration_ccdf_batch_tokens_baseline_{model_to_plot}_conv130.csv"))
splitwise_df.to_csv(os.path.join(plotsdata_dir, f"duration_ccdf_batch_tokens_splitwise_{model_to_plot}_conv130.csv"))
#plt.savefig(os.path.join(plots_dir, f"duration_ccdf_batch_tokens_{model_to_plot}_conv70.pdf"))
plt.savefig(os.path.join(plots_dir, f"duration_ccdf_batch_tokens_{model_to_plot}_conv130.pdf"))
plt.show()
plt.close()

In [ ]:
def plot_sweep(results_df, title=None, x="num_prompts"):
    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(6, 4), sharex=True, constrained_layout=True)
    sns.lineplot(data=results_df, x=x, y="ttft_slowdown_p90", style="system", markers=True, markersize=7, ax=axs[0][0])
    sns.lineplot(data=results_df, x=x, y="tbt_slowdown_p90", style="system", markers=True, markersize=7, ax=axs[1][0])
    sns.lineplot(data=results_df, x=x, y="e2e_slowdown_p90", style="system", markers=True, markersize=7, ax=axs[2][0])
    sns.lineplot(data=results_df, x=x, y="ttft_slowdown_p50", style="system", markers=True, markersize=7, ax=axs[0][1])
    sns.lineplot(data=results_df, x=x, y="tbt_slowdown_p50", style="system", markers=True, markersize=7, ax=axs[1][1])
    sns.lineplot(data=results_df, x=x, y="e2e_slowdown_p50", style="system", markers=True, markersize=7, ax=axs[2][1])

    for ax in axs.flatten():
        ax.grid()
        ax.set_xlabel(f"{x} servers")

    #sns.move_legend(axs[0][0], "lower center", bbox_to_anchor=(0.5, 1.05), ncol=2, title="")
    axs[0][0].get_legend().set_visible(False)
    axs[1][0].get_legend().set_visible(False)
    axs[2][0].get_legend().set_visible(False)
    axs[0][1].get_legend().set_visible(False)
    axs[1][1].get_legend().set_visible(False)
    axs[2][1].get_legend().set_visible(False)

    # create a single legend in center of figure
    handles, labels = axs[0][0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=3, title="", bbox_to_anchor=(0.5, 1.01))

    #axs[0][0].set_yscale("log")
    #axs[0][1].set_yscale("log")
    #axs[2][0].set_yscale("log")
    #axs[2][1].set_yscale("log")
    #axs[0][0].set_ylim(bottom=0, top=2)
    #axs[0][1].set_ylim(bottom=0, top=2)

    # set same y axis limits for plots in the same row (same as 0th column)
    #axs[0][1].set_ylim(axs[0][0].get_ylim())
    #axs[1][1].set_ylim(axs[1][0].get_ylim())
    #axs[2][1].set_ylim(axs[2][0].get_ylim())


    axs[0][0].set_ylabel("p90 TTFT (s)")
    axs[1][0].set_ylabel("p90 TBT (s)")
    #axs[2][0].set_ylabel("p99 Nth Token (s)")
    axs[2][0].set_ylabel("p90 e2e (s)")
    axs[0][1].set_ylabel("p50 TTFT (s)")
    axs[1][1].set_ylabel("p50 TBT (s)")
    axs[2][1].set_ylabel("p50 e2e (s)")
    #axs[2][1].set_ylabel("p50 Nth Token (s)")

    if title:
        fig.suptitle(title, fontsize=12)

    # set 300dpi
    plt.gcf().set_dpi(300)

    plt.show()
    plt.close()


In [ ]:
# isopower sweeps

configs = []

max_h100 = 40
for num_h100 in range(5, max_h100, 5):
    #num_a100 = int(10200 * (max_h100 - num_h100) // 6500)
    num_a100 = int(44 * (max_h100 - num_h100) // 24.8)
    configs.append(splitwise_ha_config(num_h100, num_a100))

for num_h100 in range(5, max_h100, 5):
    num_prompts = num_h100
    num_tokens = max_h100 - num_prompts
    configs.append(splitwise_hh_config(num_prompts, num_tokens))
    configs.append(splitwise_hhcap_config(num_prompts, num_tokens))

max_a100 = 70
for num_prompts in range(5, max_a100, 5):
    num_tokens = max_a100 - num_prompts
    configs.append(splitwise_aa_config(num_prompts, num_tokens))

#traces = ["rr_conv_70"]
traces = ["rr_code_100"]

results_df, request_dfs = get_data(configs, traces, seed=seed, model=model_to_plot)
plot_sweep(results_df, title=f"Isopower sweeps, {traces[0]}")

In [ ]:
# isocost sweeps

configs = []

max_h100 = 40
for num_h100 in range(5, max_h100, 5):
    #num_a100 = int(10200 * (max_h100 - num_h100) // 6500)
    num_a100 = int(4.76 * (max_h100 - num_h100) // 2.21)
    configs.append(splitwise_ha_config(num_h100, num_a100))

for num_h100 in range(5, max_h100, 5):
    num_prompts = num_h100
    num_tokens = max_h100 - num_prompts
    configs.append(splitwise_hh_config(num_prompts, num_tokens))

max_a100 = 86
for num_tokens in range(5, max_a100, 5):
    num_prompts = max_a100 - num_tokens
    configs.append(splitwise_aa_config(num_prompts, num_tokens))

#traces = ["rr_conv_70"]
traces = ["rr_code_90"]

results_df, request_dfs = get_data(configs, traces, seed=seed, model=model_to_plot)
plot_sweep(results_df, title=f"Isocost sweeps, {traces[0]}")